In [1]:
import numpy as np

In [2]:
def Normalization(nd, x):
    '''
    Normalization of coordinates (centroid to the origin and mean distance of sqrt(2 or 3).

    Input
    -----
    nd: number of dimensions, 3 here
    x: the data to be normalized (directions at different columns and points at rows)
    Output
    ------
    Tr: the transformation matrix (translation plus scaling)
    x: the transformed data
    '''

    x = np.asarray(x)
    m, s = np.mean(x, 0), np.std(x)
    if nd == 2:
        Tr = np.array([[s, 0, m[0]], [0, s, m[1]], [0, 0, 1]])
    else:
        Tr = np.array([[s, 0, 0, m[0]], [0, s, 0, m[1]], [0, 0, s, m[2]], [0, 0, 0, 1]])
        
    Tr = np.linalg.inv(Tr)
    x = np.dot( Tr, np.concatenate( (x.T, np.ones((1,x.shape[0]))) ) )
    x = x[0:nd, :].T

    return Tr, x


In [3]:
def DLTcalib(nd, xyz, uv):
    '''
    Camera calibration by DLT using known object points and their image points.

    Input
    -----
    nd: dimensions of the object space, 3 here.
    xyz: coordinates in the object 3D space.
    uv: coordinates in the image 2D space.

    The coordinates (x,y,z and u,v) are given as columns and the different points as rows.

    There must be at least 6 calibration points for the 3D DLT.

    Output
    ------
     L: array of 11 parameters of the calibration matrix.
     err: error of the DLT (mean residual of the DLT transformation in units of camera coordinates).
    '''
    if (nd != 3):
        raise ValueError('%dD DLT unsupported.' %(nd))
    
    # Converting all variables to numpy array
    xyz = np.asarray(xyz)
    uv = np.asarray(uv)

    n = xyz.shape[0]

    # Validating the parameters:
    if uv.shape[0] != n:
        raise ValueError('Object (%d points) and image (%d points) have different number of points.' %(n, uv.shape[0]))

    if (xyz.shape[1] != 3):
        raise ValueError('Incorrect number of coordinates (%d) for %dD DLT (it should be %d).' %(xyz.shape[1],nd,nd))

    if (n < 6):
        raise ValueError('%dD DLT requires at least %d calibration points. Only %d points were entered.' %(nd, 2*nd, n))
        
    # Normalize the data to improve the DLT quality (DLT is dependent of the system of coordinates).
    # This is relevant when there is a considerable perspective distortion.
    # Normalization: mean position at origin and mean distance equals to 1 at each direction.
    Txyz, xyzn = Normalization(nd, xyz)
    Tuv, uvn = Normalization(2, uv)

    A = []

    for i in range(n):
        x, y, z = xyzn[i, 0], xyzn[i, 1], xyzn[i, 2]
        u, v = uvn[i, 0], uvn[i, 1]
        A.append( [x, y, z, 1, 0, 0, 0, 0, -u * x, -u * y, -u * z, -u] )
        A.append( [0, 0, 0, 0, x, y, z, 1, -v * x, -v * y, -v * z, -v] )

    # Convert A to array
    A = np.asarray(A) 

    # Find the 11 parameters:
    U, S, V = np.linalg.svd(A)

    # The parameters are in the last line of Vh and normalize them
    L = V[-1, :] / V[-1, -1]
    #print(L)
    # Camera projection matrix
    H = L.reshape(3, nd + 1)
    #print(H)

    # Denormalization
    # pinv: Moore-Penrose pseudo-inverse of a matrix, generalized inverse of a matrix using its SVD
    H = np.dot( np.dot( np.linalg.pinv(Tuv), H ), Txyz )
    #print(H)
    H = H / H[-1, -1]
    print(H)
    #print(H)
    L = H.flatten()
    #print(L)

    # Mean error of the DLT (mean residual of the DLT transformation in units of camera coordinates):
    uv2 = np.dot( H, np.concatenate( (xyz.T, np.ones((1, xyz.shape[0]))) ) ) 
    #print(uv2) 
    print(xyz)
    uv2 = uv2 / uv2[2, :]
    # Mean distance:
    err = np.sqrt( np.mean(np.sum( (uv2[0:2, :].T - uv)**2, 1)) ) 

    return L, err

In [6]:
def DLT():
    # Known 3D coordinates
    #xyz = [[-875, 0, 9.755], [442, 0, 9.755], [1921, 0, 9.755], [2951, 0.5, 9.755], [-4132, 0.5, 23.618],
    #[-876, 0, 23.618]]
    # Known pixel coordinates
    #uv = [[76, 706], [702, 706], [1440, 706], [1867, 706], [264, 523], [625, 523]]
    uv  = [[140,285],[908,338],[1655,391],[2450,431],[3223,465],[4070,535],[4945,552],[200,1045],[941,1112],[1682,1159],
           [2456,1232],[3243,1265],[4064,1326],[4878,1392],[661,2100],[1435,2166],[2222,2233],[3043,2306],[3877,2380],
          [4744,2460],[334,2400],[1148,2460],[1995,2547],[2849,2627],[3737,2713],[4658,2794],[834,2807],[1722,2907],[2629,2987]
          ,[3570,3080],[4551,3167],[474,3214],[1408,3314],[2389,3401],[3377,3488],[4437,3614]]
    
    ### Definging World Points
    X = [216,180,144,108,72,36,0,216,180,144,108,72,36,0,180,144,108,72,36,0,180,144,108,72,36,0,144,108,72,36,0,144,108,72,36,0]
    Y = [72,72,72,72,72,72,72,36,36,36,36,36,36,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    Z = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,36,36,36,36,36,36,72,72,72,72,72,72,108,108,108,108,108,144,144,144,144,144]

    n = (len(X))
    xyz = []
    for i in range(n):
        xyz.append([X[i],Y[i],Z[i]])



    nd = 3
    P, err = DLTcalib(nd, xyz, uv)
    print('Matrix')
    print(P)
    print('\nError')
    print(err)

In [7]:
if __name__ == "__main__":
    DLT()

[[-2.11261894e+01 -1.51313720e+00 -9.71752065e+00  4.82500913e+03]
 [-1.09663690e+00 -2.24127540e+01  4.44521351e+00  2.15769756e+03]
 [ 3.61190741e-04 -6.11386943e-04 -1.57212250e-03  1.00000000e+00]]
[[216  72   0]
 [180  72   0]
 [144  72   0]
 [108  72   0]
 [ 72  72   0]
 [ 36  72   0]
 [  0  72   0]
 [216  36   0]
 [180  36   0]
 [144  36   0]
 [108  36   0]
 [ 72  36   0]
 [ 36  36   0]
 [  0  36   0]
 [180   0  36]
 [144   0  36]
 [108   0  36]
 [ 72   0  36]
 [ 36   0  36]
 [  0   0  36]
 [180   0  72]
 [144   0  72]
 [108   0  72]
 [ 72   0  72]
 [ 36   0  72]
 [  0   0  72]
 [144   0 108]
 [108   0 108]
 [ 72   0 108]
 [ 36   0 108]
 [  0   0 108]
 [144   0 144]
 [108   0 144]
 [ 72   0 144]
 [ 36   0 144]
 [  0   0 144]]
Matrix
[-2.11261894e+01 -1.51313720e+00 -9.71752065e+00  4.82500913e+03
 -1.09663690e+00 -2.24127540e+01  4.44521351e+00  2.15769756e+03
  3.61190741e-04 -6.11386943e-04 -1.57212250e-03  1.00000000e+00]

Error
12.605321766429839
